In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# !wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119

# Download dataset

# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
data_file = 'data/cora_full.npz'
# data_file = 'data/reddit.npz'

split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.25
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
topk=64
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

In [7]:
import warnings
warnings.filterwarnings("ignore")

# Load the data

In [8]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print(nc)

print('shape attibute matrix: ', attr_matrix.n_rows)
print('Training: ', len(train_idx))
print('Validation: ', len(val_idx))
print('Testing: ', len(test_idx))

print('train_idx: ', train_idx)

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

70
shape attibute matrix:  18800
Training:  1400
Validation:  14000
Testing:  3400
train_idx:  [   12    16    18 ... 18790 18793 18798]
Runtime: 0.34s


# Preprocessing: Calculate PPR scores

In [9]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR

start = time.time()

topk_train, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk,
                                 normalization=ppr_normalization, S=1, gamma=1.0)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")


print('topk_train', topk_train.shape)
adj_xd = adj_matrix[train_idx]
print(adj_xd[0:0 + batch_size].shape)

ppr_topk adj matrix:  (18800, 18800)
core_numbers:  (18800,)
max core:  24
idx:  (array([  381,   402,  9096,  9097,  9098,  9105, 10394, 10397, 10399,
       10411, 12219, 12254, 12260, 12765, 12783, 13527, 13532, 13533,
       15199, 15200, 16346, 17833]),)
core_numbers[idx] (22,) [24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24]
js_weighted[i]:  [14068    67   179   119  8238    11  9973    89  4804 13747 14815  2640
  2636 18565   208   178  2614 12686   193  4919   180 12687 18478    31
    39   166   182    34  4540 13465  5448  5450  5449 12685     3   226
   120   124   133 16513 16422 12683  8251 16423   162 13331  4895 12673
    35  4685 18480    48  4859    68 18476   181 13332    55  2627    85
 18496  2537    91    12]
vals_weighted[i]:  [0.00142591 0.00125693 0.00128017 0.00197748 0.00178313 0.00204185
 0.00166787 0.00195323 0.00176659 0.00223125 0.00197791 0.00181939
 0.00182669 0.00194822 0.00249162 0.00250084 0.00217932 0.00264527
 0.00271464 0.003298

# Training: Set up model and train

In [10]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, weight_decay, dropout,
                    sparse_features=type(attr_matrix) is not np.ndarray)

# print(model.debug.shape)
# print(model.A_hat.shape)
# print(model.debug2.shape)
# print(model.logits.shape)
# print(model.batch_pprw.shape)
print(model.batch_idx.shape)
# print(model.batch_adj_matrix.shape)
# print('logits: ', model.logits.shape)
# print('weighted_logits: ', model.weighted_logits.shape)
# print('adj_weighted_logits: ', model.adj_weighted_logits.shape)

2022-06-24 11:56:11 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_82498/2823771243.py:2: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2022-06-24 11:56:11 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_82498/2823771243.py:3: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



(?,)


In [11]:
sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            topk_train=topk_train, topk_val=topk_val,
            labels=labels,
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2022-06-24 11:56:13 (INFO): Epoch 6, step 20: train 3.94861
2022-06-24 11:56:14 (INFO): Epoch 13, step 40: train 3.36426
2022-06-24 11:56:16 (INFO): Epoch 19, step 60: train 2.99155
2022-06-24 11:56:17 (INFO): Epoch 26, step 80: train 2.63591
2022-06-24 11:56:19 (INFO): Epoch 33, step 100: train 2.01826
2022-06-24 11:56:20 (INFO): Epoch 39, step 120: train 1.72956
2022-06-24 11:56:22 (INFO): Epoch 46, step 140: train 1.67018
2022-06-24 11:56:23 (INFO): Epoch 53, step 160: train 1.31234
2022-06-24 11:56:25 (INFO): Epoch 59, step 180: train 1.17071
2022-06-24 11:56:28 (INFO): Epoch 66, step 200: train 1.25415
2022-06-24 11:56:30 (INFO): Epoch 73, step 220: train 1.02107
2022-06-24 11:56:32 (INFO): Epoch 79, step 240: train 0.95567
2022-06-24 11:56:34 (INFO): Epoch 86, step 260: train 1.03369
2022-06-24 11:56:36 (INFO): Epoch 93, step 280: train 0.87443
2022-06-24 11:56:37 (INFO): Epoch 99, step 300: train 0.84731
2022-06-24 11:56:39 (INFO): Epoch 106, step 320: train 0.92425
2022-06-24 1

Runtime: 50.98s


# Inference (val and test)

In [12]:
start = time.time()
predictions, time_logits, time_propagation = model.predict(
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Runtime: 0.23s


# Collect and print results

In [13]:

acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')



#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [14]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 99.3%, val: 61.4%, test: 61.3%
F1 score: Train: 0.972, val: 0.512, test: 0.509

Runtime: Preprocessing: 38.61s, training: 50.98s, inference: 0.23s -> total: 89.82s
Memory: Main: 3.97GB

